# debugging fdtd models

In [ ]:
%matplotlib inline
import sys

import matplotlib.pyplot as plt
import numpy as np
import sax
import yaml

# matplotlib.use('macosx')

sys.path.append("/Users/vahid/Downloads/PhotonicsAI_Project")
from PhotonicsAI.KnowledgeBase.DesignLibrary import (
    _directional_coupler,
    _mmi2x2,
    mzi_2x2_heater_tin_cband,
    straight,
)

model_mmi2x2 = _mmi2x2.get_model_fdtd
model_st = straight.get_model_fdtd

mzi, info = sax.circuit(
    netlist={
        "instances": {
            "lft": "coupler",
            "top": "waveguide",
            "btm": "waveguide",
            "rgt": "coupler",
        },
        "connections": {
            "lft,o4": "btm,o1",
            "btm,o2": "rgt,o1",
            "lft,o3": "top,o1",
            "top,o2": "rgt,o2",
        },
        "ports": {
            "o1": "lft,o1",
            "o2": "lft,o2",
            "o4": "rgt,o4",
            "o3": "rgt,o3",
        },
    },
    models={
        "coupler": model_mmi2x2,
        "waveguide": model_st,
    },
)


mzi, info = sax.circuit(
    netlist=yaml.safe_load(netlist1),
    models={
        "C1": mzi_2x2_heater_tin_cband.get_model,
        "C2": _directional_coupler.get_model_fdtd,
    },
)


wl = np.linspace(1.5, 1.6, 500)
y1 = model_mmi2x2(wl=wl)[("o1", "o3")]
y2 = model_st(wl=wl)[("o1", "o2")]
y3 = mzi(wl=wl)[("o1", "o3")]


plt.figure()
plt.plot(wl, np.abs(y1), label="y1")
plt.plot(wl, np.abs(y2), label="y2")

plt.plot(wl, np.abs(y3), label="y3")
plt.legend()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

d_mmi = np.load(
    "../../KnowledgeBase/FDTD/cband/mmi2x2/mmi2x2_taper1p3_length36p2_width5p52_gap0p27.npz"
)
x = d_mmi["wavelengths"]
y = d_mmi["o1@0,o3@0"]
plt.figure()
plt.title("mmi2x2_taper1p3_length36p2_width5p52_gap0p27.npz")
plt.plot(x, (np.angle(y)))
plt.ylabel("phase o1@0,o3@0")


d_bend = np.load(
    "../../KnowledgeBase/FDTD/cband/bend_euler/bend_euler_npoints500_radius10.npz"
)
x = d_bend["wavelengths"]
y = d_bend["o1@0,o2@0"]
plt.figure()
plt.title("bend_euler_npoints500_radius10.npz")
plt.plot(x, (np.angle(y)))
plt.ylabel("phase o1@0,o2@0")


# have to unwrap before interpolating